In [144]:
# import
from tqdm.auto import tqdm
from gensim.models import Word2Vec
import pandas as pd

In [3]:
# 1. klt2000을 사용하여 명사 추출
klt2000_output_file = open('ilbe_comments.txt', 'r', encoding='utf-8')

klt2000_output_tmp = klt2000_output_file.readlines()
klt2000_output = []
tmp = []

for token in tqdm(klt2000_output_tmp):
    token_info = token.split(':')
    
    if token_info[2] == '뷁뷁뷁\n':
        klt2000_output.append(tmp)
        tmp = []
        continue
    
    if token_info[1] == 'N' or token_info[1] == 'P' or token_info[1] == '1':
        tmp.append(token_info[2].replace('\n', ''))

  0%|          | 0/12018131 [00:00<?, ?it/s]

In [4]:
# 2. kcc150으로 학습된 word2vec 모델을 불러온다.
model = Word2Vec.load('word2vec-KCC150.model')

In [126]:
# 3. word2vec에서 시드 어휘와 비슷한 유사도를 가지고 관련있는 것으로 사전 구축
hate_dict1 = ['혐오']
hate_dict2 = ['멸시']
hate_dict3 = [i[0] for i in model.wv.most_similar('욕설', topn=3)]
hate_dict4 = [i[0] for i in model.wv.most_similar('인종차별', topn=3)]
hate_dict5 = [i[0] for i in model.wv.most_similar('조롱', topn=2)]
hate_dict = []

for word in tqdm(hate_dict1):
    words = model.wv.most_similar(word, topn=3)

    for i in words:
        if i[1] > 0.705 and i[0] not in hate_dict1:
            hate_dict1.append(i[0])
        
        if i[1] <= 0.705:
            break
            
for word in tqdm(hate_dict1):
    if model.wv.similarity(hate_dict1[0], word) > 0.605 and word not in hate_dict:
        hate_dict.append(word)
        
for word in tqdm(hate_dict2):
    words = model.wv.most_similar(word, topn=3)

    for i in words:
        if i[1] > 0.705 and i[0] not in hate_dict2:
            hate_dict2.append(i[0])
        
        if i[1] <= 0.705:
            break
            
for word in tqdm(hate_dict2):
    if model.wv.similarity(hate_dict2[0], word) > 0.605 and word not in hate_dict:
        hate_dict.append(word)
        
for word in tqdm(hate_dict3):
    words = model.wv.most_similar(word, topn=3)

    for i in words:
        if i[1] > 0.705 and i[0] not in hate_dict3:
            hate_dict3.append(i[0])
        
        if i[1] <= 0.705:
            break
            
for word in tqdm(hate_dict3):
    if model.wv.similarity(hate_dict3[0], word) > 0.605 and word not in hate_dict:
        hate_dict.append(word)
        
for word in tqdm(hate_dict4):
    words = model.wv.most_similar(word, topn=2)

    for i in words:
        if i[1] > 0.705 and i[0] not in hate_dict4:
            hate_dict4.append(i[0])
        
        if i[1] <= 0.705:
            break
            
for word in tqdm(hate_dict4):
    if model.wv.similarity(hate_dict4[0], word) > 0.605 and word not in hate_dict:
        hate_dict.append(word)
        
for word in tqdm(hate_dict5):
    words = model.wv.most_similar(word, topn=3)

    for i in words:
        if i[1] > 0.705 and i[0] not in hate_dict5:
            hate_dict5.append(i[0])
        
        if i[1] <= 0.705:
            break
            
for word in tqdm(hate_dict5):
    if model.wv.similarity(hate_dict5[0], word) > 0.605 and word not in hate_dict:
        hate_dict.append(word)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/81 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/80 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/82 [00:00<?, ?it/s]

In [147]:
# 4. 모든 문장에 대해서 klt2000으로 추출된 명사를 어휘집에 있는 단어 모두와 유사도를 계산하여 합산
# 4-1. 합산된 값의 평균을 구하여 그 문장의 최종 score로 설정
output = []

for sent in tqdm(klt2000_output[:10]):
    value = 0
    
    for token in sent:
        if token in model.wv.index_to_key:
            for hate in hate_dict:
                value += model.wv.similarity(hate, token)
                
    if len(sent) == 0:
        output.append(0)
    else:
        output.append(value/len(sent))

  0%|          | 0/10 [00:00<?, ?it/s]

In [140]:
# 4-2. 계산된 값 저장
import pickle

with open('output_tmp.pkl', 'wb') as f:
    pickle.dump(output, f)

In [149]:
# 4-3. 계산된 값 복구
with open('output_tmp.pkl', 'rb') as f:
    output = pickle.load(f)

In [145]:
# 5. 최종 score가 결정된 약 100 ~ 200개의 문장을 sampling하여 score의 기준을 설정
ilbe_c = pd.read_csv('ilbe_comments.csv')

for i, j in zip(output[:100], ilbe_c['댓글'].tolist()[:100]):
    print(i, ' / ', j)


4.799934431289633  /  좀더 시간이 흘러야 진실이 혹은 진실에 가깝게 밝혀질듯
2.8215604719516705  /  근데 나이 60에 저정도원한이면 보통감정이쌓인게 아닌건 확실함은퇴하고 조용히 여생보낼준비할때인데 것도 임원출신이 칼부림이라니 막장인생칼부림하곤 무게가 다름
1.564551204707151  /  ㄹㅇ 시발 임원출신이면 노후걱정없이 ㅆㅅㅌㅊ로 살다 자연사할운명인데그런 이점이 모두안보일정도로 이성을 잃은듯싶다 ㅋㅋㅋㅋ
4.333185963332653  /  죽여도 무죄
1.1894375147065148  /  근데 은행권이 성희롱 성추행 전나심함 ㅋㅋ 전에 사귄여친이 은행원이였는데 듣고 놀랐다 이기야
1.8612180322874337  /  증권쪽도ㅇㅇㅇㅇ그래서 사직하고 백수로 스폰이나 하고다니는여자봄. 외모 ㅅㅌㅊ이고 외국계 다녔는데
0.4348881607875228  /  존나 특이한케이스네
1.2542885219946684  /  그집아빠가 딸보고 일하지말라고 했다고함아빠는 작은사업체 운영하고있음 지금은 여자나이가 30중반일걸백수된게 10년쯤전
2.335157385458135  /  은행권 애들 직장내 성희랑 장난아님.2차 노래방가면 브루스추고 난리 아니라더라.
0.07017005015707885  /  ㅇㅇ 성희롱 성추행 사례모음집 보면 거의다 은행권에서 나온거
2.854197633396185  /  오늘 죽은 사람 : 6년전에 가해자가 성범죄 저질렀디고 허위 고발 ⟩ 재판결과 무죄⟩ 가해자 복직⟩ 가해자가 소송 진행하면서 쓴 변호사비 공금횡령이라 시비털음⟩ 가해자가 빡쳐서 범행이라고 적혀있네
0.535825179395033  /  그냥 ^대구^ 한마디로 요약가능
3.6574683315702714  /  모든 인간은 칼 앞에 평등하다 ㄷㄷ
2.016412189346738  /  무죄 땅 땅! 크
3.049783449143029  /  원래 국민이 재판, 처벌의 힘을 국가에 맡기는게 사적으로 처벌하는 거보다 공정하고확실한 일

In [56]:
# 6. 추가적으로 문장을 sampling하여 score 기준에 대한 정확도 검사 후 모든 문장에 대해서 이 방법을 진행

In [151]:
hate_dict

['혐오',
 '증오',
 '경멸',
 '증오심',
 '멸시',
 '모멸',
 '적대감',
 '혐오감',
 '죄의식',
 '열등감',
 '천대',
 '핍박',
 '냉대',
 '푸대접',
 '폭언',
 '막말',
 '욕설',
 '인종차별적',
 '인종차별',
 '모욕',
 '모독',
 '모멸감',
 '모욕감',
 '굴욕감',
 '수치심']